<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/kubeflow/kubeflow_setup_cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reading Google Cloud credentials from drive

In [0]:
import configparser
import os
from google.colab import drive

drive.mount('/content/drive')

GCP_INI = '/content/drive/My Drive/gcp/gcp.ini'
config = configparser.ConfigParser()
config.read(GCP_INI)
print('Setting environment variables:')
for key in config['credentials']:
  print(f'- {key.upper()}')
  os.environ[key.upper()] = config['credentials'][key]

In [2]:
#@title ## Setup Kubectl
#@markdown Downloads the lastest `kubectl` utility.
%%shell
curl -LO -s https://storage.googleapis.com/kubernetes-release/release/`curl -s https://storage.googleapis.com/kubernetes-release/release/stable.txt`/bin/linux/amd64/kubectl
chmod +x ./kubectl
sudo mv ./kubectl /usr/local/bin/kubectl
kubectl version --client

Client Version: version.Info{Major:"1", Minor:"18", GitVersion:"v1.18.1", GitCommit:"7879fc12a63337efff607952a323df90cdc7a335", GitTreeState:"clean", BuildDate:"2020-04-08T17:38:50Z", GoVersion:"go1.13.9", Compiler:"gc", Platform:"linux/amd64"}


In [0]:
#@title ## Setup Goggle Cloud SDK
#@markdown Downloads the lastest `gcloud` command interface.
%%shell

# Add the Cloud SDK distribution URI as a package source
echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] http://packages.cloud.google.com/apt cloud-sdk main" | sudo tee -a /etc/apt/sources.list.d/google-cloud-sdk.list

# Import the Google Cloud Platform public key
curl -s https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key --keyring /usr/share/keyrings/cloud.google.gpg add -

# Update the package list and install the Cloud SDK
sudo apt-get update -y -qq && sudo apt-get install google-cloud-sdk -qq -y

## Configure Google Cloud project

In [0]:
%env PROJECT=data-centaur-273800
%env ZONE=us-west1-b

In [0]:
!gcloud init --console-only

In [0]:
%%shell
gcloud config set project ${PROJECT} 
gcloud config set compute/zone ${ZONE}

In [0]:
!gcloud auth application-default login

In [0]:
!gcloud projects list

## Setup `kfctl` command utility

In [0]:
%env CONFIG_URI=https://raw.githubusercontent.com/kubeflow/manifests/v1.0-branch/kfdef/kfctl_gcp_iap.v1.0.1.yaml

In [0]:
%env KF_NAME=kfmnist
%env BASE_DIR=kf_deployments
%env KF_DIR=/content/kf_deployments/kfmnist
%env KF_DIR_PARENT=/content/kf_deployments

Clean up previous configuration (if any)

In [0]:
%cd /content/
!rm -rf ${KF_DIR_PARENT}

In [0]:
%%shell
wget https://github.com/kubeflow/kfctl/releases/download/v1.0.1/kfctl_v1.0.1-0-gf3edb9b_linux.tar.gz -q
tar -xvf kfctl_v1.0.1-0-gf3edb9b_linux.tar.gz
/content/kfctl version
mkdir -p ${KF_DIR}

In [0]:
%cd {os.environ['KF_DIR']}

In [0]:
!/content/kfctl apply -V -f ${CONFIG_URI}

In [0]:
!gcloud container clusters get-credentials ${KF_NAME} --zone ${ZONE} --project ${PROJECT}

In [0]:
!kubectl -n kubeflow get all

## Access the Kubeflow user interface (UI)

`https://kfmnist.endpoints.data-centaur-273800.cloud.goog`

In [0]:
%cd /content/
!kubectl -n istio-system get ingress

In [0]:
!kubectl -n istio-system get ingress envoy-ingress -o=jsonpath={.spec.rules[0].host}

In [0]:
%env HOST=kfmnist.endpoints.data-centaur-273800.cloud.goog

## Deleting Deployed Cluster

In [0]:
%%shell
sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys CC86BB64
sudo add-apt-repository ppa:rmescandon/yq
sudo apt update
sudo apt install yq -y

In [0]:
!echo ${KF_DIR}

In [0]:
%env CONFIG_FILE={KF_DIR}/kfctl_gcp_iap.v1.0.1.yaml

In [0]:
!echo ${CONFIG_FILE}

In [0]:
%%shell
yq w -i ${CONFIG_FILE} 'spec.plugins[0].spec.project' ${PROJECT}
yq w -i ${CONFIG_FILE} 'spec.plugins[0].spec.zone' ${ZONE}
yq w -i ${CONFIG_FILE} 'metadata.name' ${KF_NAME}

In [0]:
!./kfctl delete -f ${CONFIG_FILE} --delete_storage

In [0]:
!./kfctl delete -f ${CONFIG_FILE}

In [0]:
!kubectl get pods --all-namespaces